# 4. Hierarchically Chaining the Agent Calls using a Router Agent

We will now create a hierarchical workflow using a router agent. Instead of having a fixed linear workflow like in the previous notebook - `3-Wrap-SmolAgent-and-MCP-into-ACP`, we will use a fourth agent (router) in the client-side that will decide when to call each ACP agent.

## 4.1. Start Up both ACP Servers

First make sure the Travel Package ACP server and Travel Guidance ACP Servers are still running:
- If the agents are still running from the previous notebooks, then we don't need to do anything else.
- If the agents have stopped running, then we can run the servers again by typing:
  - `uv run python src/crew_agent_server.py`
  - `uv run python src/smolagents_server.py`

## 4.2. Import ACPCallingAgent

The router agent is already implemented as the `ACPCallingAgent`. Please check a python file called `fastacp.py` where we can find the definition of the `ACPCallingAgent`. 

In [1]:
import sys
import os

sys.path.append(os.path.abspath("../travel_acp_project/src"))

In [2]:
import asyncio 
import nest_asyncio
from acp_sdk.client import Client
from smolagents import LiteLLMModel
from fastacp import AgentCollection, ACPCallingAgent
from colorama import Fore

c:\Vinayak\anaconda3\envs\acp-agents\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(ACPCallingAgent.__doc__)


    This agent uses JSON-like ACP agent calls, similarly to how ToolCallingAgent uses tool calls,
    but directed at remote ACP agents instead of local tools.
    
    Args:
        acp_agents (`dict[str, Agent]`): ACP agents that this agent can call.
        model (`Callable[[list[dict[str, str]]], ChatMessage]`): Model that will generate the agent's actions.
        prompt_templates ([`Dict[str, str]`], *optional*): Prompt templates.
        planning_interval (`int`, *optional*): Interval at which the agent will run a planning step.
        **kwargs: Additional keyword arguments.
    


## 4.3. Run the Hierarchical Workflow 

In [4]:
nest_asyncio.apply()

**Note**: The `fastacp.py` file does not only contain the definition for the ACPCallingAgent, but it also includes this method: `AgentCollection.from_acp` where the client objects (`travel_package_agent`, `travel_research_agent` and `travel_agency_agent`) discover the agents hosted on their corresponding servers by calling the method `.agents()`.

In [5]:
model = LiteLLMModel(
    model_id="openai/gpt-4"
)

async def run_travel_workflow() -> None:
    # Step 1: Connect to both ACP servers
    async with Client(base_url="http://localhost:8001") as package, Client(base_url="http://localhost:8000") as guidance:
        
        # Step 2: Discover all agents from both servers
        agent_collection = await AgentCollection.from_acp(package, guidance)
        acp_agents = {
            agent.name: {'agent': agent, 'client': client}
            for client, agent in agent_collection.agents
        }

        print(Fore.CYAN + "Discovered ACP Agents:\n" + "\n".join(acp_agents.keys()) + Fore.RESET)

        # Step 3: Initialize the Router Agent (ACPCallingAgent)
        router = ACPCallingAgent(acp_agents=acp_agents, model=model)

        # Step 4: Run the full query
        query = (
            "I’m based in Hyderabad. Which travel agencies can help me? "
            "When is the best time to visit Europe? "
            "Are there any packages from Hyderabad that include Switzerland?"
        )

        result = await router.run(query)
        print(Fore.YELLOW + f"\nFinal result:\n{result}" + Fore.RESET)

In [6]:
asyncio.run(run_travel_workflow())

Discovered ACP Agents:
travel_package_agent
travel_research_agent
travel_agency_agent
[INFO] Step 1/10
[DEBUG] Output message of the LLM:
[DEBUG] ModelResponse(id='chatcmpl-ByuqlxOuD6TIXNmTvzWPfrpc1po1O', created=1753857911, model='gpt-4-0613', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='tool_calls', index=0, message=Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{\n  "prompt": "Find travel agencies in Hyderabad"\n}', name='travel_agency_agent'), id='call_3JkLMFTPmtamN3abFbNWHB1S', type='function')], function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=19, prompt_tokens=562, total_tokens=581, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDe

c:\Vinayak\anaconda3\envs\acp-agents\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content=None, rol...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='to...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


run_id=UUID('88632a42-2736-475a-88f0-9a796c2b171c') agent_name='travel_agency_agent' session_id=UUID('1942be60-893e-4c65-b46f-017aabd2edb1') status=<RunStatus.COMPLETED: 'completed'> await_request=None output=[Message(role='agent/travel_agency_agent', parts=[MessagePart(name=None, content_type='text/plain', content='Travel agencies in Hyderabad:\n1. GT Holidays Hyderabad - Rating: 4.7 - Contact: https://www.gtholidays.in/hyderabad/ - Packages: Domestic, International, Europe\n2. RL Tours & Travels - Rating: 4.5 - Contact: https://rltours.in/ - Packages: Custom Europe, Asia, Domestic\n3. IMAD Travel - Rating: No rating available - Contact: https://imadtravel.com/ - Packages: Luxury International Packages, Umrah, Europe\n4. StarEdge Holidays - Rating: 4.6 - Packages: International Packages, Group Tours\n5. Thrillophilia - Rating: No rating available - Contact: https://www.thrillophilia.com - Packages: Adventure Experiences, Multi-day Tours, International Packages', content_encoding='plai

run_id=UUID('f88a89c8-b48e-476d-a65a-45341d31fcae') agent_name='travel_research_agent' session_id=UUID('1bad31c9-81ff-45ed-a2bf-ec00578a2f75') status=<RunStatus.COMPLETED: 'completed'> await_request=None output=[Message(role='agent/travel_research_agent', parts=[MessagePart(name=None, content_type='text/plain', content='The best time to visit Europe depends on several factors such as the specific destination, personal weather, crowd and budget preferences. However, for a mix of good weather, lower prices, and manageable crowds, the spring (April–June) and autumn (September–October) seasons are commonly suggested.', content_encoding='plain', content_url=None, metadata=None)], created_at=datetime.datetime(2025, 7, 30, 6, 45, 55, 608273, tzinfo=TzInfo(UTC)), completed_at=datetime.datetime(2025, 7, 30, 6, 45, 55, 608277, tzinfo=TzInfo(UTC)))] error=None created_at=datetime.datetime(2025, 7, 30, 6, 45, 30, 176131, tzinfo=TzInfo(UTC)) finished_at=datetime.datetime(2025, 7, 30, 6, 45, 55, 610

run_id=UUID('87013922-9f41-4458-b76d-510d0fb2dfed') agent_name='travel_package_agent' session_id=UUID('a2b50ecd-41e6-4a74-bafd-6e59472026b2') status=<RunStatus.COMPLETED: 'completed'> await_request=None output=[Message(role='agent/travel_package_agent', parts=[MessagePart(name=None, content_type='text/plain', content='Here are some travel packages from Hyderabad that include Switzerland:\n\n1. "Best of Switzerland & Italy | Roman Streets to Snowy Alps" - 9 days & 8 nights for INR 1,43,200\n2. "Scandinavian Snow Escape | Northern Nights & Winter Lights" - 9 days & 8 nights for INR 2,91,000\n3. "Europe Golden Trio | Switzerland, Amsterdam & Paris Tour" - 8 days & 7 nights for INR 1,73,000\n4. "Classic Europe in Winter | From Roman Ruins to Amsterdam Canals" - 13 days & 12 nights for INR 2,55,000\n5. "Western Europe In A Nutshell" - 11 days & 10 nights for INR 1,72,000\n6. "Classic Switzerland | Peaks, Lakes, and Alpine Trails" - 7 days & 6 nights for INR 1,25,900\n\nAll packages are offe

**Optional Reading:** Here's how the hierarchical flow works using the provided file `fastacp.py`:

<img src="./images/Hierarchical_Workflow.jpg">

1. The agents hosted on each server are first discovered by their corresponding client objects and then converted to tools for the router agent (ACPCallingAgent): 

2. When the router agent receives a user query, it breaks downs the query into smaller steps where each step can be executed by the specialized agent. For a given step, the router agent uses the client of the specialized agent to send the request to it:

    **🧭 Step 1: Travel Agency Agent (MCP Tool - in Guidance ACP)**

    **Input:** "I am based in Hyderabad. Are there any travel agencies near me?"

    **Output:**

      - GT Holidays, RL Tours & Travels, IMAD Travel, StarEdge Holidays, Thrillophilia (✅ as seen in your logs)

    **🧭 Step 2: Travel Research Agent (Smolagent - in Guidance ACP)**
    
    **Input:**

      - "Based on these agencies: {above list}, What is the best time to go for a Europe trip from Hyderabad?"

    **Output:**

      - “Spring (April–June) and Autumn (Sept–Oct) are best. Weather is good, prices are better, and fewer crowds.”

    **🧭 Step 3: Travel Package Agent (CrewAI + RAG - in Travel Package ACP)**

    **Input:**

      - Context: {agencies + season insight}

      - "Which Europe travel packages from Hyderabad include Switzerland?"

3. **🤖 Role of the Router Agent.** `ACPCallingAgent` behaves like an intelligent coordinator:

    🧠 Analyzes the user’s query and context.

    📌 Breaks it into 3 sub-goals: Agency lookup → Research → Package details.

    🪄 Sequentially calls:

    `travel_agency_agent` → via MCP

    `travel_research_agent` → via SmolAgent

    `travel_package_agent` → via CrewAI/RAG

    It accumulates results in memory (as seen with Saved to memory: `travel_package_agent_response=`), and finally calls `final_answer` to compose the structured answer.